<a href="https://colab.research.google.com/github/regiiis/dlv_group_project/blob/main/object_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>